# Trading Dashboard

---

### Import Libraries and Dependencies

In [15]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot
import hvplot.pandas
from IPython.display import Markdown

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

### Read CSV into Pandas DataFrame

In [16]:
# Set the file path
filepath = Path("../Resources/aapl.csv")

# Read the CSV located at the file path into a Pandas DataFrame
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,date,close,volume,open,high,low
0,9/22/14,101.06,52421660,101.80,102.14,100.58
1,9/23/14,102.64,63255860,100.60,102.94,100.54
2,9/24/14,101.75,59974260,102.16,102.85,101.20
3,9/25/14,97.87,99689300,100.51,100.71,97.72
4,9/26/14,100.75,62276770,98.53,100.75,98.40


In [33]:
(aapl_df.iloc[-1]['close']-aapl_df.iloc[0]['close'])/aapl_df.iloc[0]['close']

1.1864239065901445

### Generate a Dual Moving Average Crossover Trading Signal

In [17]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = aapl_df.loc[:, ['date', 'close']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("date", drop=True)

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
9/6/19,213.260,205.2584,199.72930,1.0,0.0
9/9/19,214.170,205.5470,199.87850,1.0,0.0
9/10/19,216.700,205.9226,200.01420,1.0,0.0
9/11/19,223.590,206.3634,200.21150,1.0,0.0
9/12/19,223.085,206.7705,200.39705,1.0,0.0
9/13/19,218.750,207.0573,200.50975,1.0,0.0
9/16/19,219.900,207.3707,200.63715,1.0,0.0
9/17/19,220.700,207.7843,200.79135,1.0,0.0
9/18/19,222.770,208.2149,200.97605,1.0,0.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [18]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

### Backtest the Trading Strategy

In [19]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
9/6/19,213.260,205.2584,199.72930,1.0,0.0,500.0,0.0,106630.0,22495.0,129125.0,-0.000077,0.291250
9/9/19,214.170,205.5470,199.87850,1.0,0.0,500.0,0.0,107085.0,22495.0,129580.0,0.003524,0.295800
9/10/19,216.700,205.9226,200.01420,1.0,0.0,500.0,0.0,108350.0,22495.0,130845.0,0.009762,0.308450
9/11/19,223.590,206.3634,200.21150,1.0,0.0,500.0,0.0,111795.0,22495.0,134290.0,0.026329,0.342900
9/12/19,223.085,206.7705,200.39705,1.0,0.0,500.0,0.0,111542.5,22495.0,134037.5,-0.001880,0.340375
9/13/19,218.750,207.0573,200.50975,1.0,0.0,500.0,0.0,109375.0,22495.0,131870.0,-0.016171,0.318700
9/16/19,219.900,207.3707,200.63715,1.0,0.0,500.0,0.0,109950.0,22495.0,132445.0,0.004360,0.324450
9/17/19,220.700,207.7843,200.79135,1.0,0.0,500.0,0.0,110350.0,22495.0,132845.0,0.003020,0.328450
9/18/19,222.770,208.2149,200.97605,1.0,0.0,500.0,0.0,111385.0,22495.0,133880.0,0.007791,0.338800


### Plot Entry/Exit Points of Trading Strategy vs. Backtest Results

In [20]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [21]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [22]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.0666828
Cumulative Returns,0.32975
Annual Volatility,0.137751
Sharpe Ratio,0.484084
Sortino Ratio,0.685956


### Prepare Trade Evaluation Metrics DataFrame

In [23]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Generating Trade Evaluation Metrics

In [24]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2/12/15,8/3/15,500.0,126.46,118.44,63230.0,59220.0,4010.0
1,AAPL,12/9/15,1/13/16,500.0,115.62,97.39,57810.0,48695.0,9115.0
2,AAPL,4/25/16,6/14/16,500.0,105.08,97.46,52540.0,48730.0,3810.0
3,AAPL,8/22/16,3/26/18,500.0,108.51,172.77,54255.0,86385.0,-32130.0
4,AAPL,4/19/18,11/30/18,500.0,172.80,178.58,86400.0,89290.0,-2890.0
5,AAPL,3/28/19,7/22/19,500.0,188.72,207.22,94360.0,103610.0,-9250.0


---

### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [25]:
price_df = signals_df[['close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

### Create Hvplot Table of Portfolio Metrics

In [26]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

### Create Hvplot Table of Trade Metrics

In [27]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

### Build a Dashboard with `hvplot`

In [28]:
# Assemble dashboard visualization
display(Markdown("# Trading Dashboard"))
display(price_chart)
portfolio_evaluation_table + trade_evaluation_table

# Trading Dashboard

:NdOverlay   [Variable]
   :Curve   [date]   (value)

:Layout
   .Table.I  :Table   [index,Backtest]
   .Table.II :Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]